In [1]:
import csv
import sqlite3
import functions
import pandas as pd
import requests as rq
from config import api_key
# pip install geopy
from geopy.geocoders import Nominatim
import time
from math import ceil
import pickle

In [2]:
# Loading model to compare the results ### model = pickle.load(open('model.pkl','rb'))
df = pickle.load(open('df.pkl','rb'))
df

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Title Polarity,Title Subjectivity,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,0.600,0.9,True,False,False,False,False,False,False,True
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,-1.000,1.0,False,True,False,False,False,False,False,True
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,0.000,0.0,False,True,False,False,False,False,True,False
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.300,0.3,False,True,False,False,False,False,False,True
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.700,0.6,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,Korean Air,6,2016,5,4,5,3,4,7,1,...,0.000,0.0,False,True,False,False,True,False,False,False
8096,Korean Air,6,2016,3,5,5,4,5,10,1,...,0.000,0.0,False,True,False,False,False,True,False,False
8097,Korean Air,4,2016,4,5,5,5,1,2,0,...,0.000,0.0,True,False,False,False,True,False,False,False
8098,Korean Air,4,2016,5,1,3,4,5,10,1,...,0.200,0.1,True,False,False,False,True,False,False,False


In [3]:
print(f'Rows and Columns: {df.shape}')
df.columns

Rows and Columns: (7683, 31)


Index(['Airline', 'Month Flown', 'Year Flown', 'Seat Comfort', 'Staff Service',
       'Food & Beverages', 'Inflight Entertainment', 'Value For Money',
       'Overall Rating', 'Recommended', 'Origin', 'Destination', 'Via',
       'Origin_Code', 'Destination_Code', 'Via_Code', 'Origin_City',
       'Destination_City', 'Via_City', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure'],
      dtype='object')

In [4]:
unique_codes_Origin = df['Origin_Code'].unique()
len(unique_codes_Origin)
unique_codes_Origin

array(['CGK', 'HKT', 'SAI', 'SIN', 'BWN', 'LHR', 'PER', 'SYD', 'MIL',
       'LON', 'BKK', 'KUL', 'PEN', 'FRA', 'BNE', 'MEL', 'NYC', 'AKL',
       'JNB', 'MNL', 'BOM', 'ATH', 'CAN', 'KCZ', 'DXB', 'SFO', 'ADL',
       'CPT', 'EWR', 'HKG', 'YVR', 'PNH', 'DVO', 'DPS', 'HAN', 'MAN',
       'JFK', 'LAX', 'NRT', 'IST', 'SUB', 'PAR', 'BLR', 'AMS', 'SGN',
       'DAD', 'ROM', 'CCU', 'MUC', 'CMB', 'SAO', 'KIX', 'HND', 'ZRH',
       'UPG', 'AMD', 'CPH', 'PUS', 'SPK', 'PVG', 'DEL', 'DAC', 'SEA',
       'ARN', 'CHC', 'ICN', 'DUS', 'WLG', 'BCN', 'TPE', 'RGN', 'CJB',
       'MAA', 'IAH', 'CBR', 'DME', 'MXP', 'CDG', 'FCO', 'DOH', 'ORD',
       'NCE', 'TBS', 'BEY', 'LGW', 'DFW', 'KWI', 'LUN', 'BRU', 'RUH',
       'OSL', 'KTM', 'VIE', 'BWI', 'AUH', 'EBB', 'PHL', 'WAW', 'EDI',
       'ISB', 'GVA', 'YMQ', 'DAL', 'MPM', 'BUH', 'DUB', 'HRE', 'BOS',
       'MAD', 'IKA', 'SOF', 'LOS', 'PRG', 'NBO', 'ATL', 'ALG', 'SHJ',
       'MIA', 'ZAG', 'COK', 'ESB', 'KGL', 'BSR', 'MLE', 'YTO', 'TUN',
       'CAI', 'BJV',

In [5]:
unique_codes_Dest = df['Destination_Code'].unique()
len(unique_codes_Dest)
unique_codes_Dest

array(['SIN', 'LHR', 'PNH', 'CNS', 'FRA', 'BWN', 'CGK', 'HND', 'HKG',
       'IST', 'BKK', 'MNL', 'MEL', 'ADL', 'MAN', 'SYD', 'AKL', 'PER',
       'PEN', 'ZAG', 'DPS', 'LAX', 'JFK', 'DXB', 'HKT', 'FUK', 'CHC',
       'PAR', 'NYC', 'DEL', 'ZRH', 'KIX', 'SGN', 'LON', 'MIL', 'SEA',
       'JNB', 'KUL', 'BNE', 'CPH', 'IAH', 'SFO', 'YTO', 'AMS', 'CPT',
       'BOM', 'MUC', 'CCU', 'SAI', 'EWR', 'MLE', 'CBU', 'SVO', 'DUB',
       'NRT', 'TPE', 'NGO', 'USM', 'UPG', 'CMB', 'ICN', 'HAN', 'CJB',
       'TAO', 'PVG', 'TRV', 'BLR', 'KCZ', 'CBR', 'SUB', 'PEK', 'ATH',
       'ROM', 'WLG', 'DUS', 'DAC', 'ARN', 'XMN', 'AMD', 'CAN', 'BCN',
       'MAA', 'VTE', 'CDG', 'CNX', 'DME', 'GRU', 'RGN', 'MXP', 'YMQ',
       'DOH', 'JRO', 'DET', 'BSR', 'GOI', 'BRU', 'WAW', 'SLL', 'TBS',
       'LYS', 'BWI', 'LAD', 'LUN', 'KWI', 'SAO', 'ORD', 'LGW', 'KTM',
       'MCT', 'VIE', 'BAH', 'AUH', 'EDI', 'IKA', 'BER', 'WDH', 'PHL',
       'RUH', 'LOS', 'COK', 'BGW', 'BUH', 'DAL', 'MAD', 'NBO', 'SHJ',
       'ISB', 'PRG',

In [6]:
unique_codes_Origin = set(unique_codes_Origin)
unique_codes_Dest = set(unique_codes_Dest)
unique_codes = unique_codes_Origin.union(unique_codes_Dest)
unique_codes

{'ABJ',
 'ABV',
 'ABZ',
 'ACC',
 'ADB',
 'ADD',
 'ADL',
 'AER',
 'AEX',
 'AGP',
 'AJA',
 'AKL',
 'ALA',
 'ALG',
 'AMD',
 'AMM',
 'AMS',
 'AOJ',
 'AQJ',
 'ARN',
 'ASR',
 'ATH',
 'ATL',
 'ATQ',
 'AUH',
 'AUS',
 'AXT',
 'AYT',
 'BAH',
 'BAK',
 'BCN',
 'BEG',
 'BER',
 'BEY',
 'BGO',
 'BGW',
 'BHM',
 'BHX',
 'BIA',
 'BIO',
 'BIQ',
 'BJL',
 'BJV',
 'BKK',
 'BKO',
 'BLL',
 'BLQ',
 'BLR',
 'BNE',
 'BOD',
 'BOG',
 'BOM',
 'BOS',
 'BRE',
 'BRU',
 'BSL',
 'BSR',
 'BUD',
 'BUE',
 'BUH',
 'BUS',
 'BWI',
 'BWN',
 'BXH',
 'CAI',
 'CAN',
 'CAS',
 'CAY',
 'CBR',
 'CBU',
 'CCJ',
 'CCS',
 'CCU',
 'CDG',
 'CEB',
 'CGK',
 'CGN',
 'CHC',
 'CJB',
 'CJU',
 'CKG',
 'CKY',
 'CLJ',
 'CMB',
 'CMN',
 'CND',
 'CNS',
 'CNX',
 'COK',
 'CPH',
 'CPT',
 'CRK',
 'CTA',
 'CTS',
 'CTU',
 'CUN',
 'CVG',
 'CWL',
 'DAC',
 'DAD',
 'DAL',
 'DAR',
 'DBV',
 'DEL',
 'DEN',
 'DET',
 'DFW',
 'DKR',
 'DLA',
 'DLC',
 'DLM',
 'DME',
 'DMM',
 'DOH',
 'DPS',
 'DTW',
 'DUB',
 'DUR',
 'DUS',
 'DVO',
 'DWC',
 'DXB',
 'DYU',
 'EAP',
 'EBB',


In [7]:
unique_codes_list = list(unique_codes)
unique_codes_list

['IST',
 'RIO',
 'BJV',
 'PNH',
 'SEA',
 'LGK',
 'ARN',
 'HAN',
 'CWL',
 'TSA',
 'EVN',
 'WLG',
 'FLR',
 'DUR',
 'IAH',
 'NUE',
 'OKA',
 'MED',
 'KTM',
 'SEL',
 'BXH',
 'LYP',
 'PUS',
 'NQZ',
 'KBP',
 'SKP',
 'DKR',
 'SJO',
 'RGN',
 'CHC',
 'LSG',
 'KRK',
 'OZH',
 'SXF',
 'HND',
 'BLL',
 'MSP',
 'SJC',
 'YTO',
 'NCE',
 'AGP',
 'GRU',
 'YMQ',
 'VLC',
 'QUF',
 'AER',
 'ALA',
 'BHX',
 'LYS',
 'GLA',
 'FUK',
 'PMI',
 'MAD',
 'BIQ',
 'AMM',
 'ZNZ',
 'JNB',
 'GUW',
 'GYD',
 'DEL',
 'NAG',
 'YYZ',
 'DYU',
 'NGO',
 'TIA',
 'MUC',
 'DAC',
 'SIN',
 'IKA',
 'SHJ',
 'RBA',
 'SXM',
 'TLV',
 'RMO',
 'IZM',
 'CCS',
 'SEZ',
 'SKT',
 'LHR',
 'KCZ',
 'BEY',
 'KGL',
 'AQJ',
 'MPM',
 'GMP',
 'AJA',
 'RZV',
 'IWK',
 'OSL',
 'GZP',
 'UEE',
 'MGQ',
 'DVO',
 'ABJ',
 'PRN',
 'SYD',
 'ORD',
 'TLS',
 'NAN',
 'OKJ',
 'WDH',
 'SCL',
 'BSL',
 'MVD',
 'MCO',
 'HRB',
 'EZS',
 'VNO',
 'WAW',
 'SGN',
 'SLC',
 'ALG',
 'IAD',
 'ODS',
 'PVG',
 'SAI',
 'EDI',
 'GVA',
 'KEF',
 'DPS',
 'CND',
 'PUJ',
 'FLL',
 'SHE',
 'HAV',


In [8]:
len(unique_codes_list)

424

In [9]:
custom_user_agent = "MyCustomUserAgent"
geolocator = Nominatim(user_agent="custom_user_agent")

In [10]:
geolocator.geocode('MEX airport').latitude

19.434325299999998

In [11]:
def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(f"{airport_code} airport")
    print(airport_code, location)
    time.sleep(1)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

In [12]:
def split_unique_codes_list(unique_codes_list, num_splits):
    elements_per_split = ceil(len(unique_codes_list) / num_splits)
    split_lists = [unique_codes_list[i:i + elements_per_split] for i in range(0, len(unique_codes_list), elements_per_split)]

    return split_lists

num_splits = 10

result = split_unique_codes_list(unique_codes_list, num_splits)

# Assigning split lists to unique variables
for i, sublist in enumerate(result):
    globals()[f"split_{i+1}"] = sublist

# Printing the unique variables
for i in range(10):
    print(f"split_{i+1}: {globals()[f'split_{i+1}']}")

split_1: ['IST', 'RIO', 'BJV', 'PNH', 'SEA', 'LGK', 'ARN', 'HAN', 'CWL', 'TSA', 'EVN', 'WLG', 'FLR', 'DUR', 'IAH', 'NUE', 'OKA', 'MED', 'KTM', 'SEL', 'BXH', 'LYP', 'PUS', 'NQZ', 'KBP', 'SKP', 'DKR', 'SJO', 'RGN', 'CHC', 'LSG', 'KRK', 'OZH', 'SXF', 'HND', 'BLL', 'MSP', 'SJC', 'YTO', 'NCE', 'AGP', 'GRU', 'YMQ']
split_2: ['VLC', 'QUF', 'AER', 'ALA', 'BHX', 'LYS', 'GLA', 'FUK', 'PMI', 'MAD', 'BIQ', 'AMM', 'ZNZ', 'JNB', 'GUW', 'GYD', 'DEL', 'NAG', 'YYZ', 'DYU', 'NGO', 'TIA', 'MUC', 'DAC', 'SIN', 'IKA', 'SHJ', 'RBA', 'SXM', 'TLV', 'RMO', 'IZM', 'CCS', 'SEZ', 'SKT', 'LHR', 'KCZ', 'BEY', 'KGL', 'AQJ', 'MPM', 'GMP', 'AJA']
split_3: ['RZV', 'IWK', 'OSL', 'GZP', 'UEE', 'MGQ', 'DVO', 'ABJ', 'PRN', 'SYD', 'ORD', 'TLS', 'NAN', 'OKJ', 'WDH', 'SCL', 'BSL', 'MVD', 'MCO', 'HRB', 'EZS', 'VNO', 'WAW', 'SGN', 'SLC', 'ALG', 'IAD', 'ODS', 'PVG', 'SAI', 'EDI', 'GVA', 'KEF', 'DPS', 'CND', 'PUJ', 'FLL', 'SHE', 'HAV', 'ESB', 'CPH', 'EWR', 'HKD']
split_4: ['NRT', 'NCL', 'UPG', 'PEW', 'BSR', 'AUH', 'FRA', 'ZRH', '

In [13]:
print(split_1)

['IST', 'RIO', 'BJV', 'PNH', 'SEA', 'LGK', 'ARN', 'HAN', 'CWL', 'TSA', 'EVN', 'WLG', 'FLR', 'DUR', 'IAH', 'NUE', 'OKA', 'MED', 'KTM', 'SEL', 'BXH', 'LYP', 'PUS', 'NQZ', 'KBP', 'SKP', 'DKR', 'SJO', 'RGN', 'CHC', 'LSG', 'KRK', 'OZH', 'SXF', 'HND', 'BLL', 'MSP', 'SJC', 'YTO', 'NCE', 'AGP', 'GRU', 'YMQ']


In [14]:
results = [get_lat_long(airport_code) for airport_code in split_1]

IST İstanbul Havalimanı, Vecihi Hürkuş Caddesi, Tayakadın Mahallesi, Arnavutköy, İstanbul, Marmara Bölgesi, 34277, Türkiye
RIO None
BJV Milas-Bodrum Havalimanı, Milas Havalimanı Yolu, Ekinanbarı, Milas, Muğla, Ege Bölgesi, 48670, Türkiye
PNH អាកាសយានដ្ឋាន​អន្តរជាតិ​ភ្នំពេញ, ផ្លូវបេតុង, សាលាសង្កាត់ត្រពាងំក្រសាំង, សង្កាត់កាកាបទី២, ខណ្ឌពោធិ៍សែនជ័យ, រាជធានីភ្នំពេញ, 120912, ព្រះរាជាណាចក្រ​កម្ពុជា
SEA Seattle-Tacoma International Airport, Air Cargo Road, SeaTac, King County, Washington, 98158, United States
LGK Lapangan Terbang Antarabangsa Langkawi, Jalan Lapangan Terbang Langkawi 2, Padang Matsirat, Langkawi, Kedah, 07100, Malaysia
ARN Stockholm-Arlanda flygplats, Radarvägen, Benstocken, Sigtuna kommun, Stockholms län, 190 60, Sverige
HAN Sân bay quốc tế Nội Bài, Đường Võ Nguyên Giáp, Xã Phú Cường, Huyện Sóc Sơn, Thành phố Hà Nội, Việt Nam
CWL Cardiff Airport, Porthkerry Road, Lower Porthkerry, Rhoose, Vale of Glamorgan, Cymru / Wales, CF62 3DP, United Kingdom
TSA 臺北松山機場, 340-9, 敦化北路, 精忠里,

In [34]:
df_1 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_1['Airport Code'] = split_1
df_1.shape

(43, 3)

In [17]:
print(split_2)

['VLC', 'QUF', 'AER', 'ALA', 'BHX', 'LYS', 'GLA', 'FUK', 'PMI', 'MAD', 'BIQ', 'AMM', 'ZNZ', 'JNB', 'GUW', 'GYD', 'DEL', 'NAG', 'YYZ', 'DYU', 'NGO', 'TIA', 'MUC', 'DAC', 'SIN', 'IKA', 'SHJ', 'RBA', 'SXM', 'TLV', 'RMO', 'IZM', 'CCS', 'SEZ', 'SKT', 'LHR', 'KCZ', 'BEY', 'KGL', 'AQJ', 'MPM', 'GMP', 'AJA']


In [35]:
results = [get_lat_long(airport_code) for airport_code in split_2]

VLC Aeropuerto Internacional de Valencia, Parque Fluvial del Turia, Manises, l'Horta Sud, València / Valencia, Comunitat Valenciana, 46940, España
QUF None
AER Сочи, А-149, Адлерский внутригородской район, Сочи, городской округ Сочи, Краснодарский край, Южный федеральный округ, 354383, Россия


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=ALA+airport&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [37]:
df_2 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_2['Airport Code'] = split_2
df_2

,Latitude,Longitude,Airport Code
0,NaN,NaN,VLC
1,39.049906,-84.665157,QUF
2,24.887796,55.168796,AER
3,48.688422,9.205393,ALA
4,38.749627,-90.370470,BHX
5,36.234091,59.641867,LYS
6,41.815391,12.226485,GLA
7,49.810575,23.957444,FUK
8,42.743036,2.870327,PMI
9,41.609391,41.600614,MAD


In [21]:
results = [get_lat_long(airport_code) for airport_code in split_3]

RZV Rize-Artvin Havalimanı, Havaalanı Caddesi, İkiztepe, Pazar, Rize, Karadeniz Bölgesi, 53300, Türkiye
IWK 岩国錦帯橋空港, 岩国錦帯橋空港線, 旭町二丁目, 岩国市, 山口県, 740-8511, 日本
OSL Oslo lufthavn, Gardermoen, Gardermovegen, Kneppefeltet, Søndre Børke, Nannestad, Viken, 2060, Norge
GZP Gazipaşa-Alanya Havalimanı, Paşabey Sokak, Cumhuriyet, Sarıağaç, Gazipaşa, Antalya, Akdeniz Bölgesi, 07900, Türkiye
UEE Queenstown Airport, Lyell Highway, Queenstown, West Coast, Tasmania, 7467, Australia
MGQ Garoonka Caalamiga Ee Aadan Cadde مطار آدم عدي الدولي, Danwadaag Street, Wadajir وداجير مدينة, Banaadir بنادر, Soomaaliya الصومال
DVO Francisco Bangoy International Airport, Saint Peter Street, Doña Pilar Village, Sasa, Davao City, Davao Region, 8000, Pilipinas
ABJ Aéroport International Félix Houphouët-Boigny, Rue Clément Ader, Adjoufou, Port-Bouët, Abidjan, Côte d’Ivoire
PRN Aeroporti Ndërkombëtar i Prishtinës "Adem Jashari", Aeroporti, Vrellë, Vrellë e Goleshit, Komuna e Lipjanit / Opština Lipljan, Rajoni i Prishtinës

In [38]:
df_3 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_3['Airport Code'] = split_3
df_3

,Latitude,Longitude,Airport Code
0,NaN,NaN,RZV
1,39.049906,-84.665157,IWK
2,24.887796,55.168796,OSL
3,48.688422,9.205393,GZP
4,38.749627,-90.370470,UEE
5,36.234091,59.641867,MGQ
6,41.815391,12.226485,DVO
7,49.810575,23.957444,ABJ
8,42.743036,2.870327,PRN
9,41.609391,41.600614,SYD


In [19]:
results = [get_lat_long(airport_code) for airport_code in split_4]

NRT 成田国際空港, 成田市, 千葉県, 282-0004, 日本
NCL Newcastle International Airport, Premium Meet & Greet Drop Off Bay, Woolsington, Newcastle upon Tyne, North of Tyne, England, NE13 8BZ, United Kingdom
UPG Bandar Udara Internasional Sultan Hasanuddin, Jalan BTN Laikang Indah, Bakung, Maros, Makassar, Sulawesi Selatan, Sulawesi, 90242, Indonesia
PEW باچا خان انٹرنیشنل ائرپورٹ, سرکلر روڈ, یونیورسٹی ٹاؤن, تحصیل پشاور شہر, ضلع پشاور‎, پشاور ڈویژن, خیبر پختونخوا, 25000, پاکستان
BSR مطار البصرة الدولى, الاربعين كيلو, ناحية الهارثة, قضاء البصرة, العراق
AUH مطار زايد الدولي, شارع 6, قرية معاصرة, مطار زايد الدولي, أبو ظبي, أبوظبي, أبو ظبي, الإمارات العربية المتحدة
FRA Flughafen Frankfurt am Main, Rüsselsheim am Main, Kreis Groß-Gerau, Hessen, 60549, Deutschland
ZRH Flughafen Zürich, Rohrholzstrasse, Glattbrugg, Opfikon, Bezirk Bülach, Zürich, 8058, Schweiz/Suisse/Svizzera/Svizra
LHE علامہ اقبال انٹرنیشنل ائیرپورٹ‎, اسٹریٹ 3, Sector E, عسکری 10, گُجر پُورہ, ضلع لاہور, لاہور ڈویژن, پنجاب, 54840, پاکستان
DLA 

In [24]:
df_4 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_4['Airport Code'] = split_4
df_4

,Latitude,Longitude,Airport Code
0,41.179327,40.850241,NRT
1,34.159093,132.235929,NCL
2,60.197865,11.099675,UPG
3,36.298806,32.297182,PEW
4,-42.074782,145.525647,BSR
5,2.014088,45.304034,AUH
6,7.124880,125.643458,FRA
7,5.264928,-3.925957,ZRH
8,42.576691,21.032355,LHE
9,-33.949894,151.181968,DLA


In [25]:
results = [get_lat_long(airport_code) for airport_code in split_5]

ORN Aéroport International Ahmed Ben Bella, RN 2A, Es Senia ⵙⵙⴰⵏⵉⵢⴰ السانية, Daïra Es Senia, Oran, 31071, Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر
ECN Ercan Uluslararası Havalimanı, LF.27, Meriç, Değirmenlik-Akıncılar Belediyesi, Lefkoşa, Kuzey Kıbrıs, 99100, Κύπρος - Kıbrıs
CNS Cairns Airport, Hamilton Street, Aeroglen, Cairns, Cairns Regional, Queensland, 4870, Australia
SHA 上海虹桥国际机场, 航新路, 新虹街道, 七宝镇, 闵行区, 上海市, 201101, 中国
TAO 青岛胶东国际机场, 青岛新机场高速, 胶州市, 青岛市, 山东省, 266300, 中国
BCN Aeroport Barcelona - el Prat Josep Tarradellas, Carrer de la Rosa dels Vents, el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España
HRK HRK, проспект Гагаріна, Слобідський район, Харків, Харківська міська громада, Харківський район, Харківська область, 61031, Україна
PEN Lapangan Terbang Antarabangsa Pulau Pinang, Persiaran Bayan Lepas, Zon Perindustrian Bebas Bayan Lepas, Barat Daya, George Town, Pulau Pinang, 11900, Malaysia
DMM مطار الملك فهد الدولي "مطار الدمام", طريق مطار ارامكو, محافظة الدمام, المنطقة الشر

In [26]:
df_5 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_5['Airport Code'] = split_5
df_5

,Latitude,Longitude,Airport Code
0,35.620567,-0.622524,ORN
1,35.151752,33.503532,ECN
2,-16.877789,145.751550,CNS
3,31.197678,121.334483,SHA
4,36.361963,120.087995,TAO
5,41.296944,2.079047,BCN
6,49.922622,36.288430,HRK
7,5.296732,100.274786,PEN
8,26.467397,49.798878,DMM
9,22.312599,113.917300,HKG


In [27]:
results = [get_lat_long(airport_code) for airport_code in split_6]

CBU None
CVG Cincinnati/Northern Kentucky International Airport, Lincoln Road, Boone County, Kentucky, 41025, United States
DWC مطار آل مكتوم الدولي, DWC Peripheral road, مدينة المطار, دبي, الإمارات العربية المتحدة
STR Flughafen Stuttgart, Flughafenstraße, Plieningen, Stuttgart, Filderstadt, Baden-Württemberg, 70629, Deutschland
STL St. Louis Lambert International Airport, 10701, Lambert International Boulevard, Saint Louis, Saint Louis County, Missouri, 63145, United States
MHD فرودگاه بین‌المللی شهید هاشمی‌نژاد مشهد, سحر ۴۴, انقلاب, شهرک ابوذر, منطقه ۷, شهر مشهد, بخش مرکزی شهرستان مشهد, شهرستان مشهد, استان خراسان رضوی, 91639-33345, ایران
FCO Aeroporto di Roma-Fiumicino, Via Leonardo da Vinci, Fiumicino, Roma Capitale, Lazio, 00054, Italia
LWO Міжнародний аеропорт «Львів» ім. Данила Галицького, вулиця Івана Франка, Границя, Березівка, Скнилів, Зимноводівська сільська громада, Львівський район, Львівська область, 81110, Україна
PGF Aéroport de Perpignan - Rivesaltes, D 117, Rivesaltes,

In [28]:
df_6 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_6['Airport Code'] = split_6
df_6

,Latitude,Longitude,Airport Code
0,NaN,NaN,CBU
1,39.049906,-84.665157,CVG
2,24.887796,55.168796,DWC
3,48.688422,9.205393,STR
4,38.749627,-90.370470,STL
5,36.234091,59.641867,MHD
6,41.815391,12.226485,FCO
7,49.810575,23.957444,LWO
8,42.743036,2.870327,PGF
9,41.609391,41.600614,BUS


In [29]:
results = [get_lat_long(airport_code) for airport_code in split_7]

SKG Διεθνής Κρατικός Αερολιμένας Θεσσαλονίκης "Μακεδονία", Θεσσαλονίκης - Περαίας, Νέο Ρύσιο, Δήμος Θέρμης, Μητροπολιτική Ενότητα Θεσσαλονίκης, Περιφέρεια Κεντρικής Μακεδονίας, Αποκεντρωμένη Διοίκηση Μακεδονίας - Θράκης, 570 01, Ελλάς
PEK 北京首都国际机场, 四纬路, 半壁店村, 朝阳区, 北京市, 100621, 中国
CAY Aéroport de Cayenne - Félix Éboué, Route de Rochambeau, Matoury, Cayenne, Guyane, 97351, France
IEV Міжнародний аеропорт «Київ» (Жуляни) імені Ігоря Сікорського, Промениста вулиця, ЖК ParkLand, Солом’янський район, Київ, 03066, Україна
RNS Rennes Aéroport Bretagne, Rue Jacqueline Auriol, La Chevrolais, La Morinais, Saint-Jacques-de-la-Lande, Rennes, Ille-et-Vilaine, Bretagne, France métropolitaine, 35091, France
JED مطار الملك عبد العزيز الدولي, محافظة جدة, منطقة مكة المكرمة, 21474, السعودية
KRT مطار الخرطوم الدولي, Mamoon Al Berar Street, امتداد ناصر, بري, الخرطوم, ولاية الخرطوم, 00249, السودان
MPL Aéroport de Montpellier - Méditerranée, Avenue Louis Pasteur, Les Galfières, Pérols, Montpellier, Hérault, O

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=PAR+airport&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [30]:
df_7 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_7['Airport Code'] = split_7
df_7

,Latitude,Longitude,Airport Code
0,NaN,NaN,SKG
1,39.049906,-84.665157,PEK
2,24.887796,55.168796,CAY
3,48.688422,9.205393,IEV
4,38.749627,-90.370470,RNS
5,36.234091,59.641867,JED
6,41.815391,12.226485,KRT
7,49.810575,23.957444,MPL
8,42.743036,2.870327,ATQ
9,41.609391,41.600614,LIS


In [31]:
results = [get_lat_long(airport_code) for airport_code in split_8]

KBL میدان هوایی بین‌المللی حامد کرزی, روسی روڈ, ناحیه پانزدهم, کابل, کابل ښاروالۍ, ولایت كابل, 1007, افغانستان
YXE Saskatoon John G. Diefenbaker International Airport, Claypool Drive, Saskatoon, Division No. 11, Saskatchewan, S7R 0M8, Canada
YVR YVR–Airport, Walkway to Canada Line, Sea Island, Richmond, Metro Vancouver Regional District, British Columbia, V7B 1W0, Canada
DTW Detroit Metropolitan Airport, Romulus, Wayne County, Michigan, 48174, United States
HTY Hatay Havaalanı, Havaalanı yolu, Serinyol, Paşaköy, Antakya, Hatay, Akdeniz Bölgesi, 31115, Türkiye
JIB Aéroport International de Djibouti-Ambouli مطار جيبوتي الدولي, RN-2, Djibouti جيبوتي, Arta عرتا, Djibouti جيبوتي
TZX Trabzon Uluslararası Havalimanı, Pelitli Gözde Sokak, Yeşilköy Mahallesi, Trabzon, Ortahisar, Trabzon, Karadeniz Bölgesi, 61010, Türkiye
SAV Savannah/Hilton Head International Airport, Robert B Miller Road, Savannah, Chatham County, Georgia, 31418, United States
TOY Toy, Mayom, Unity, South Sudan جنوب السودان
UI

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=DAL+airport&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [32]:
df_8 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_8['Airport Code'] = split_8
df_8

,Latitude,Longitude,Airport Code
0,NaN,NaN,KBL
1,39.049906,-84.665157,YXE
2,24.887796,55.168796,YVR
3,48.688422,9.205393,DTW
4,38.749627,-90.370470,HTY
5,36.234091,59.641867,JIB
6,41.815391,12.226485,TZX
7,49.810575,23.957444,SAV
8,42.743036,2.870327,TOY
9,41.609391,41.600614,UIO


In [33]:
results = [get_lat_long(airport_code) for airport_code in split_9]

LED Пулково, Вертолётная улица, Авиагородок, округ Пулковский меридиан, Санкт-Петербург, Северо-Западный федеральный округ, 196210, Россия
BKK คิง พาวเวอร์, ถนนสุวรรณภูมิ 1, ตำบลหนองปรือ, องค์การบริหารส่วนตำบลหนองปรือ, อำเภอบางพลี, จังหวัดสมุทรปราการ, 10520, ประเทศไทย
DFW Dallas/Fort Worth International Airport, 2400, Aviation Drive, Dallas, Tarrant County, Texas, 75261, United States
VAN Van Ferit Melen Havalimanı, Ipek Yolu, Süphan Mahallesi, Edremit, Van, Doğu Anadolu Bölgesi, 65300, Türkiye
OGU Ordu Giresun Havalimanı, Ordu-Giresun yolu, Ürümbey Mahallesi, Gülistan Mahallesi, Gülyalı, Ordu, Karadeniz Bölgesi, 52010, Türkiye
GOT Göteborg Landvetter Airport, Flygplatsvägen, Härryda kommun, Västra Götalands län, 438 80, Sverige
ABV Nnamdi Azikiwe International Airport, AVM Femi John Femi Road, Municipal Area Council, Federal Capital Territory, Nigeria
YEA None
EAP EuroAirport Bâle-Mulhouse-Fribourg, Rue de la Barrière, Ritty, Saint-Louis, Mulhouse, Haut-Rhin, Grand Est, France métropo

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=VIE+airport&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [39]:
df_9 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])
df_9['Airport Code'] = split_9
df_9

,Latitude,Longitude,Airport Code
0,NaN,NaN,LED
1,39.049906,-84.665157,BKK
2,24.887796,55.168796,DFW
3,48.688422,9.205393,VAN
4,38.749627,-90.370470,OGU
5,36.234091,59.641867,GOT
6,41.815391,12.226485,ABV
7,49.810575,23.957444,YEA
8,42.743036,2.870327,EAP
9,41.609391,41.600614,SVG


In [42]:
results = [get_lat_long(airport_code) for airport_code in split_10]

df_10 = pd.DataFrame(results, columns=['Latitude', 'Longitude'])

# Check the length of the DataFrame and split_10
if len(df_10) != len(split_10):
    print("Length of DataFrame and split_10 do not match.")
    # Adjust the length of split_10 or handle the mismatch accordingly

# Assign Airport Code to the DataFrame
df_10['Airport Code'] = split_10

# Display the DataFrame
print(df_10)

CMB Bandaranaike International Airport, Katunayake-Veyangoda Road, Katunayake, Gampaha District, Western Province, 11550, ශ්‍රී ලංකාව இலங்கை
AMS Luchthaven Schiphol, Badhoevedorp, Haarlemmermeer, Noord-Holland, Nederland, 1171 VM, Nederland
MAN Aéroport de Man, Route de l'Aéroport, Tonkpi, Montagnes, Côte d’Ivoire
CGN Flughafen Köln/Bonn, 12, Heinrich-Steinmann-Straße, Grengel, Porz, Köln, Nordrhein-Westfalen, 51147, Deutschland
BLQ Aeroporto di Bologna "Guglielmo Marconi", Via Persicetana Vecchia, Borgo Panigale, Borgo Panigale-Reno, Bologna, Emilia-Romagna, 40132, Italia
MRU Sir Seewoosagur Ramgoolam International Airport, Route Royale Plaine Magnien – Mahebourg, Beau Vallon, Grand Port, 50810, Mauritius / Maurice
TRV Thiruvananthapuram International Airport, pettah bridge, Pettah, Thiruvananthapuram, Thiruvananthapuram District, Kerala, 695001, India
DET Coleman A. Young Municipal Airport, East McNichols Road, LaSalle College Park, Detroit, Wayne County, Michigan, 48205, United Stat

In [43]:
df_10

,Latitude,Longitude,Airport Code
0,7.178959,79.883819,CMB
1,52.326980,4.741505,AMS
2,7.271981,-7.589400,MAN
3,50.867782,7.138961,CGN
4,44.534431,11.288457,BLQ
5,-20.432561,57.684949,MRU
6,8.482042,76.919059,TRV
7,42.411244,-83.008531,DET
8,44.568339,26.102534,OTP
9,23.392003,113.301182,CAN


In [46]:
airport_positions_df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10], ignore_index=True)
airport_positions_df.shape

(424, 3)

In [47]:
airport_positions_df.to_csv('Resources/airport_positions.csv', index=False)